In [ ]:
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots


import pandas as pd
import sqlite3
from pathlib import Path
from datetime import datetime, timedelta, timezone

import matplotlib.pyplot as plt
from wordcloud import WordCloud
import jieba

tz = timezone(timedelta(hours=8))

APPLICATION_START = datetime(2026, 2, 1, 19, 0, 0, tzinfo=tz)

DATABASE_PATH = Path("db.sqlite3")
db = sqlite3.connect(DATABASE_PATH)
cursor = db.cursor()
plotly_figures = {}

# Read database tables
applicants = pd.read_sql_query("SELECT * FROM applicant", db)
wechat_infos = pd.read_sql_query("SELECT * FROM wechat_info", db)

# Analyse signup trends

In [ ]:
BIN = "10min"


app_created = pd.to_datetime(applicants["created_at"], utc=True, errors="coerce").dt.tz_convert(tz)
wc_created = pd.to_datetime(wechat_infos["created_at"], utc=True, errors="coerce").dt.tz_convert(tz)

# Define plotting window
start = pd.Timestamp(APPLICATION_START).tz_convert(tz).floor(BIN)
end = pd.Timestamp(datetime.now(tz)).tz_convert(tz).ceil(BIN)
idx = pd.date_range(start=start, end=end, freq=BIN, tz=tz)

# Per-bin counts
app_bin = app_created.dt.floor(BIN).value_counts().sort_index().reindex(idx, fill_value=0)
wc_bin = wc_created.dt.floor(BIN).value_counts().sort_index().reindex(idx, fill_value=0)

# Cumulative
app_cum = app_bin.cumsum()
wc_cum = wc_bin.cumsum()

x = idx.to_pydatetime()  # plotly-friendly

# 1) Interactive line graph: per-bin counts
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=x,
        y=app_bin.values,
        mode="lines",
        name="Applications (Applicant)",
        line=dict(width=2, color="#1f77b4"),
        hovertemplate="%{x|%Y-%m-%d %H:%M}<br>Applications: %{y}<extra></extra>",
    )
)
fig.add_trace(
    go.Scatter(
        x=x,
        y=wc_bin.values,
        mode="lines",
        name="WeChat infos (WeChatInfo)",
        line=dict(width=2, color="#ff7f0e"),
        hovertemplate="%{x|%Y-%m-%d %H:%M}<br>WeChat infos: %{y}<extra></extra>",
    )
)
fig.update_layout(
    title=f"New records per {BIN}",
    xaxis_title="Time (HK)",
    yaxis_title=f"Count per {BIN}",
    hovermode="x unified",
    template="plotly_white",
    height=420,
)
fig.update_xaxes(showspikes=True, spikemode="across", spikesnap="cursor", spikedash="dot")
fig.update_yaxes(showspikes=True, spikemode="across", spikesnap="cursor", spikedash="dot")
fig.show()
plotly_figures[f"New records per {BIN}"] = fig

# 2) Interactive cumulative graph: line + area under curve
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=x,
        y=app_cum.values,
        mode="lines",
        name="Applications (cumulative)",
        line=dict(width=2, color="#1f77b4"),
        fill="tozeroy",
        fillcolor="rgba(31,119,180,0.18)",
        hovertemplate="%{x|%Y-%m-%d %H:%M}<br>Applications (cum): %{y}<extra></extra>",
    )
)
fig.add_trace(
    go.Scatter(
        x=x,
        y=wc_cum.values,
        mode="lines",
        name="WeChat infos (cumulative)",
        line=dict(width=2, color="#ff7f0e"),
        fill="tozeroy",
        fillcolor="rgba(255,127,14,0.18)",
        hovertemplate="%{x|%Y-%m-%d %H:%M}<br>WeChat infos (cum): %{y}<extra></extra>",
    )
)
fig.update_layout(
    title="Cumulative totals",
    xaxis_title="Time (HK)",
    yaxis_title="Total count",
    hovermode="x unified",
    template="plotly_white",
    height=420,
)
fig.update_xaxes(showspikes=True, spikemode="across", spikesnap="cursor", spikedash="dot")
fig.update_yaxes(showspikes=True, spikemode="across", spikesnap="cursor", spikedash="dot")
fig.show()
plotly_figures[f"Cumulative totals"] = fig

# 3) Cumulative applicants by school (overlay)
schools = applicants["school"].dropna().astype(str).str.strip()
schools = sorted(s for s in schools.unique() if s)
colors = [
    "#ff7f0e", "#2ca02c", "#d62728",
]
fill_colors = [
    "rgba(255,127,14,0.15)", "rgba(44,160,44,0.15)", "rgba(214,39,40,0.15)",
]
fig_school = go.Figure()
for i, school_name in enumerate(schools):
    mask = applicants["school"].astype(str).str.strip() == school_name
    app_created_school = pd.to_datetime(applicants.loc[mask, "created_at"], utc=True, errors="coerce").dt.tz_convert(tz)
    app_bin_school = app_created_school.dt.floor(BIN).value_counts().sort_index().reindex(idx, fill_value=0)
    app_cum_school = app_bin_school.cumsum()
    color = colors[i % len(colors)]
    fill_color = fill_colors[i % len(fill_colors)]
    fig_school.add_trace(
        go.Scatter(
            x=x,
            y=app_cum_school.values,
            mode="lines",
            name=school_name,
            line=dict(width=2, color=color),
            hovertemplate="%{x|%Y-%m-%d %H:%M}<br>" + school_name + " (cum): %{y}<extra></extra>",
            fill="tozeroy",
            fillcolor=fill_color,
        )
    )
fig_school.update_layout(
    title="Cumulative applicants by school",
    xaxis_title="Time (HK)",
    yaxis_title="Cumulative count",
    hovermode="x unified",
    template="plotly_white",
    height=420,
    legend_title="School",
)
fig_school.update_xaxes(showspikes=True, spikemode="across", spikesnap="cursor", spikedash="dot")
fig_school.update_yaxes(showspikes=True, spikemode="across", spikesnap="cursor", spikedash="dot")
fig_school.show()
plotly_figures["Cumulative applicants by school"] = fig_school

print(f"Window: {start} → {end} (HK time)")
print(f"Applicants total: {len(applicants)}")
print(f"WeChat infos total: {len(wechat_infos)}")


# Applicants info

In [ ]:
# Pie charts: sex→preferred_sex, grade, school

# 1) sex → preferred_sex (M/F only): M>M, M>F, F>M, F>F
sex_norm = applicants["sex"]
pref_norm = applicants["preferred_sex"]
pairs = (sex_norm + ">" + pref_norm).value_counts()
order = ["M>M", "M>F", "F>M", "F>F"]
pairs = pairs.reindex(order, fill_value=0)

# 2) Sex, grade and school (total) in one graph — first pie = part 1 (Sex → preferred sex)
grade_counts = applicants["grade"].value_counts()
school_counts = applicants["school"].value_counts()
fig_total = make_subplots(
    rows=1,
    cols=3,
    specs=[[{"type": "pie"}, {"type": "pie"}, {"type": "pie"}]],
    subplot_titles=("Sex → preferred sex", "Grade", "School"),
)
for i, (counts, _) in enumerate([(pairs, "Sex → preferred sex"), (grade_counts, "Grade"), (school_counts, "School")]):
    fig_total.add_trace(
        go.Pie(
            labels=counts.index.tolist(),
            values=counts.values.tolist(),
            sort=False,
            textinfo="label+percent",
            hovertemplate="%{label}<br>%{value} (%{percent})<extra></extra>",
            hole=0.35,
            showlegend=False,
        ),
        row=1,
        col=i + 1,
    )
fig_total.update_layout(
    title="Applicants total: Sex, Grade & School",
    template="plotly_white",
    height=420,
    margin=dict(t=100, b=40, l=20, r=20),
    title_y=0.98,
    title_yanchor="top",
)
# Nudge subplot titles up so they sit clear of the pie labels
for ann in fig_total.layout.annotations:
    ann.yshift = 30
fig_total.show()
plotly_figures["Applicants total: Sex, Grade & School"] = fig_total


# 3) location: HK vs Others, then distribution of Others
location = applicants["location"].astype(str).str.strip()
mask_nonempty = location.ne("")

def _is_hk(v):
    if not v:
        return False
    s = v.lower()
    return s in ("hk", "hong kong", "香港") or "hong kong" in s or "香港" in s

is_hk = location.map(_is_hk)
hk_count = (is_hk & mask_nonempty).sum()
others_count = ((~is_hk) & mask_nonempty).sum()
hk_vs_others = pd.Series({"HK": int(hk_count), "Others": int(others_count)})

others_locations = location[(~is_hk) & mask_nonempty]
others_counts = others_locations.value_counts()
if len(others_counts) == 0:
    others_counts = pd.Series({"No non-HK locations": 1})

fig_loc = make_subplots(
    rows=1,
    cols=2,
    specs=[[{"type": "pie"}, {"type": "pie"}]],
    subplot_titles=("HK vs Others", "Distribution of Others"),
)
fig_loc.add_trace(
    go.Pie(
        labels=hk_vs_others.index.tolist(),
        values=hk_vs_others.values.tolist(),
        sort=False,
        textinfo="label+percent",
        hovertemplate="%{label}<br>%{value} (%{percent})<extra></extra>",
        hole=0.35,
        showlegend=False,
    ),
    row=1,
    col=1,
)
fig_loc.add_trace(
    go.Pie(
        labels=others_counts.index.tolist(),
        values=others_counts.values.tolist(),
        sort=False,
        textinfo="label+percent",
        hovertemplate="%{label}<br>%{value} (%{percent})<extra></extra>",
        hole=0.35,
    ),
    row=1,
    col=2,
)
fig_loc.update_layout(title="Applicants by location", template="plotly_white", height=420)
fig_loc.show()
plotly_figures["Applicants by location"] = fig_loc


# Applicants info by schools

In [ ]:
# Distribution of sex and grade by school — one plot per school (sex + grade subplots)

applicants_valid = applicants[
    applicants["sex"].astype(str).str.strip().ne("")
    & applicants["grade"].astype(str).str.strip().ne("")
    & applicants["school"].astype(str).str.strip().ne("")
]
schools = applicants_valid["school"].dropna().unique().tolist()
schools = sorted(s for s in schools if str(s).strip())

for school_name in schools:
    subset = applicants_valid[applicants_valid["school"] == school_name]
    if subset.empty:
        continue
    sex_counts = subset["sex"].value_counts()
    grade_counts = subset["grade"].value_counts()

    fig = make_subplots(
        rows=1,
        cols=2,
        specs=[[{"type": "pie"}, {"type": "pie"}]],
        subplot_titles=("Sex", "Grade"),
    )
    fig.add_trace(
        go.Pie(
            labels=sex_counts.index.tolist(),
            values=sex_counts.values.tolist(),
            sort=False,
            textinfo="label+percent",
            hovertemplate="%{label}<br>%{value} (%{percent})<extra></extra>",
            hole=0.35,
            showlegend=False,
        ),
        row=1,
        col=1,
    )
    fig.add_trace(
        go.Pie(
            labels=grade_counts.index.tolist(),
            values=grade_counts.values.tolist(),
            sort=False,
            textinfo="label+percent",
            hovertemplate="%{label}<br>%{value} (%{percent})<extra></extra>",
            hole=0.35,
            showlegend=False,
        ),
        row=1,
        col=2,
    )
    title = f"Sex & grade by school — {school_name}"
    fig.update_layout(title=title, template="plotly_white", height=420)
    fig.show()
    plotly_figures[title] = fig

## MBTI info

In [ ]:
# 1) Overlayed histograms: mbti_{ei,sn,tf,jp} in [0, 100]
mbti_cols = ["mbti_ei", "mbti_sn", "mbti_tf", "mbti_jp"]

fig = go.Figure()
for col in mbti_cols:
    col_name = f"{col[-2]} - {col[-1]}".upper()
    vals = pd.to_numeric(applicants[col], errors="coerce")
    fig.add_trace(
        go.Histogram(
            x=vals,
            name=col_name,
            xbins=dict(size=3),
            opacity=0.5,
            hovertemplate=f"{col_name}<br>%{{x}}<br>count=%{{y}}<extra></extra>",
        )
    )

fig.update_layout(
    title="MBTI scores distribution (0–100) — overlayed",
    xaxis_title="Score (lower = closer to the first letter)",
    yaxis_title="Count",
    barmode="overlay",
    template="plotly_white",
    height=420
)
fig.update_xaxes(range=[0, 100])
fig.show()
plotly_figures["MBTI scores distribution (0–100) — overlayed"] = fig


# 2) MBTI preference (E/I, S/N, T/F, J/P) or X = no preference
pref_specs = {
    "preferred_mbti_ei": ["E", "I"],
    "preferred_mbti_sn": ["S", "N"],
    "preferred_mbti_tf": ["T", "F"],
    "preferred_mbti_jp": ["J", "P"],
}

rows = []
for col, letters in pref_specs.items():
    s = applicants[col].str.upper()

    counts = s.value_counts()

    order = [letters[0], "X", letters[1]]
    counts = counts.reindex([k for k in order if k in counts.index], fill_value=0)

    for k, v in counts.items():
        rows.append({"dimension": col.replace("preferred_mbti_", "").upper(), "preference": k, "count": int(v)})

pref_df = pd.DataFrame(rows)

fig = go.Figure()
for pref in ["I", "N", "F", "P", "X", "E", "S", "T", "J", ]:
    d = pref_df[pref_df["preference"] == pref]
    if d.empty:
        continue
    fig.add_trace(
        go.Bar(
            x=d["dimension"],
            y=d["count"],
            name=pref,
            opacity=0.7,
            hovertemplate=f"<br>pref: {pref}<br>count=%{{y}}<extra></extra>",
        )
    )

fig.update_layout(
    title="MBTI preference counts (X = no preference)",
    xaxis_title="Dimension",
    yaxis_title="Count",
    barmode="stack",
    template="plotly_white",
    height=420,
    showlegend=False,
)
fig.show()
plotly_figures["MBTI preference counts (X = no preference)"] = fig


# Applicant preferences

In [ ]:
# Preferred grades: bar chart — each bar = one preferred grade, stacked by applicant's grade
# preferred_grades / preferred_schools are non-empty strings with items separated by " | "

def _split_preferred(s: str, sep: str = " | "):
    if pd.isna(s) or not str(s).strip():
        return []
    return [x.strip() for x in str(s).split(sep) if x.strip()]

# 1) Preferred grades by applicant's grade
rows_grade = []
for _, row in applicants[["grade", "preferred_grades"]].iterrows():
    app_grade = row["grade"]
    if pd.isna(app_grade) or not str(app_grade).strip():
        continue
    for pref in _split_preferred(row["preferred_grades"]):
        rows_grade.append({"applicant_grade": app_grade, "preferred_grade": pref})

df_grade = pd.DataFrame(rows_grade)
if not df_grade.empty:
    cross_grade = df_grade.groupby(["preferred_grade", "applicant_grade"]).size().unstack(fill_value=0)
    grade_order = ["UG1", "UG2", "UG3", "UG4", "UG5", "MS", "PHD", "PROF", "GRAD"]
    pref_order = [g for g in grade_order if g in cross_grade.index]
    pref_order += [g for g in cross_grade.index if g not in pref_order]
    cross_grade = cross_grade.reindex(pref_order, fill_value=0)

    fig_grade = go.Figure()
    for app_grade in cross_grade.columns:
        fig_grade.add_trace(
            go.Bar(
                name=app_grade,
                x=cross_grade.index,
                y=cross_grade[app_grade],
                hovertemplate=f"Applicant grade: {app_grade}<br>Preferred grade: %{{x}}<br>Count: %{{y}}<extra></extra>",
            )
        )
    totals_grade = cross_grade.sum(axis=1)
    fig_grade.add_trace(
        go.Scatter(
            x=cross_grade.index,
            y=totals_grade,
            text=totals_grade,
            mode="text",
            textposition="top center",
            textfont=dict(size=12),
            showlegend=False,
            hoverinfo="skip",
        )
    )
    fig_grade.update_layout(
        title="Preferred grades by applicant's grade",
        xaxis_title="Preferred grade",
        yaxis_title="Count",
        barmode="stack",
        template="plotly_white",
        height=420,
        legend_title="Applicant grade",
    )
    fig_grade.show()
    plotly_figures["Preferred grades by applicant's grade"] = fig_grade
else:
    print("No preferred-grades data to plot.")

# 2) Preferred schools by applicant's school
rows_school = []
for _, row in applicants[["school", "preferred_schools"]].iterrows():
    app_school = row["school"]
    if pd.isna(app_school) or not str(app_school).strip():
        continue
    for pref in _split_preferred(row["preferred_schools"]):
        rows_school.append({"applicant_school": app_school, "preferred_school": pref})

df_school = pd.DataFrame(rows_school)
if not df_school.empty:
    cross_school = df_school.groupby(["preferred_school", "applicant_school"]).size().unstack(fill_value=0)
    school_order = ["UST", "HKU", "CU", "CUHK"]
    pref_order = [s for s in school_order if s in cross_school.index]
    pref_order += [s for s in cross_school.index if s not in pref_order]
    cross_school = cross_school.reindex(pref_order, fill_value=0)

    fig_school = go.Figure()
    for app_school in cross_school.columns:
        fig_school.add_trace(
            go.Bar(
                name=app_school,
                x=cross_school.index,
                y=cross_school[app_school],
                hovertemplate=f"Applicant school: {app_school}<br>Preferred school: %{{x}}<br>Count: %{{y}}<extra></extra>",
            )
        )
    totals_school = cross_school.sum(axis=1)
    fig_school.add_trace(
        go.Scatter(
            x=cross_school.index,
            y=totals_school,
            text=totals_school,
            mode="text",
            textposition="top center",
            textfont=dict(size=12),
            showlegend=False,
            hoverinfo="skip",
        )
    )
    fig_school.update_layout(
        title="Preferred schools by applicant's school",
        xaxis_title="Preferred school",
        yaxis_title="Count",
        barmode="stack",
        template="plotly_white",
        height=420,
        legend_title="Applicant school",
    )
    fig_school.show()
    plotly_figures["Preferred schools by applicant's school"] = fig_school
else:
    print("No preferred-schools data to plot.")

# Word Clouds

In [ ]:
# Word cloud columns:
# - hobbies, fav_movies: pipe-separated lists (" | ")
# - wish, why_lamp_remembered_your_name, weekend_arrangement, expectation: free text

import io
import base64
import matplotlib.font_manager as fm
import re

PIPE_SEP_COLS = ["hobbies", "fav_movies"]
FREE_TEXT_COLS = ["wish", "why_lamp_remembered_your_name", "weekend_arrangement", "expectation", "message_to_partner"]
TEXTS_TO_REMOVE = set([
    "因为", 
    "因为我",
    "关系",
    "我",
    "的",
    "有",
    "和",
    "人",
    "它",
    "让",
    "在",
    "是",
    "去",
    "他",
    "了",
    "都",
    "能",
    "到",
    "想",
    "很",
    "你",
    "们",
    "也",
    "就",
    "把",
    "吧"
])

# Find a font that supports Chinese (CJK) for word cloud
def _find_chinese_font():
    # Common paths by OS
    candidates = [
        "/System/Library/Fonts/Supplemental/Songti.ttc",
        "/usr/share/fonts/truetype/noto/NotoSansCJK-Regular.ttc",
        "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc",
        "/usr/share/fonts/truetype/wqy/wqy-microhei.ttc",
        "C:/Windows/Fonts/msyh.ttc",
        "C:/Windows/Fonts/simhei.ttf",
    ]
    for path in candidates:
        if Path(path).exists():
            return path
    # Search via matplotlib font_manager
    for f in fm.fontManager.ttflist:
        name = (f.name or "").lower()
        path = getattr(f, "fname", None)
        if path and ("cjk" in name or "chinese" in name or "noto sans" in name or "wenquanyi" in name or "simhei" in name or "pingfang" in name):
            return path
    return None

FONT_PATH = _find_chinese_font()
if FONT_PATH is None:
    print("Warning: No Chinese font found; word clouds may show boxes for Chinese. Install Noto Sans CJK or set font path.")


def text_from_pipe_separated(series: pd.Series, sep: str = " | ") -> str:
    """Flatten pipe-separated values into one string (each item as a 'word')."""
    parts = []
    for v in series.dropna().astype(str):
        v = v.strip()
        if not v:
            continue
        for item in v.split(sep):
            item = item.strip()
            if item:
                parts.append(item)
    return " ".join(parts)


def text_from_free_text(series: pd.Series) -> str:
    """Concatenate free text and segment with jieba for Chinese word cloud."""
    raw = " ".join(series.dropna().astype(str).str.strip())

    # remove texts to remove
    for text in TEXTS_TO_REMOVE:
        raw = raw.replace(text, "")
        
    res = jieba.cut(raw)
    str_to_return = ""
    
    for i in res:
        if i in TEXTS_TO_REMOVE:
            continue
        str_to_return += i + " "
    
    return str_to_return


def plot_word_cloud_plotly(text: str, title: str, font_path=None):
    """Generate word cloud and display as Plotly figure (for HTML export and Chinese support)."""
    if not text.strip():
        print(f"[{title}] No data, skipping.")
        return
    wc = WordCloud(
        width=1600,
        height=800,
        background_color="white",
        font_path=font_path,
        max_words=150,
        colormap="Reds",
    ).generate(text)
    # Encode as PNG and show in Plotly via layout image
    buf = io.BytesIO()
    wc.to_image().save(buf, format="png")
    buf.seek(0)
    b64 = base64.b64encode(buf.read()).decode("utf-8")
    fig = go.Figure()
    fig.add_layout_image(
        dict(
            source=f"data:image/png;base64,{b64}",
            xref="paper", yref="paper",
            x=0, y=1, sizex=1, sizey=1,
            xanchor="left", yanchor="top",
            layer="below",
        )
    )
    fig.update_xaxes(showgrid=False, zeroline=False, showticklabels=False, range=[0, 1])
    fig.update_yaxes(showgrid=False, zeroline=False, showticklabels=False, range=[0, 1])
    fig.update_layout(
        title=title,
        template="plotly_white",
        height=420,
        margin=dict(l=10, r=10, t=50, b=10),
    )
    fig.show()
    plotly_figures[title] = fig


for col in PIPE_SEP_COLS:
    text = text_from_pipe_separated(applicants[col])
    plot_word_cloud_plotly(text, f"Word cloud: {col}", font_path=FONT_PATH)

for col in FREE_TEXT_COLS:
    text = text_from_free_text(applicants[col])
    plot_word_cloud_plotly(text, f"Word cloud: {col}", font_path=FONT_PATH)

# Save the results to html

In [ ]:
out = Path("application_data_analysis.html")
html_parts = [
    "<html><head><meta charset='utf-8'></head><body>",
    "<h2>Signup analytics</h2>",
]
for name, fig in plotly_figures.items():
    html_parts.append(f"<h3>{name}</h3>")
    # include_plotlyjs='cdn' makes file smaller (needs internet to view)
    html_parts.append(pio.to_html(fig, full_html=False, include_plotlyjs="cdn"))
html_parts.append("</body></html>")

out.write_text("\n".join(html_parts), encoding="utf-8")
print("Wrote:", out.resolve())